# Inset Map of Kotlin Island

Kotlin island is situated in Gulf of Finland and is one of districts of the city of Saint Petersburg in Russia.

This example shows how Lets-Plot-Kotlin SpatialDataset integration can help to build an inset map of Kotlin island.

[The geodata is provided by © OpenStreetMap contributors and is made available here under the Open Database License (ODbL)](https://www.openstreetmap.org/copyright).

In [1]:
%useLatestDescriptors
%use klaxon
%use lets-plot
%use lets-plot-gt

In [2]:
import java.net.URL
//import org.jetbrains.letsPlot.spatial.SpatialDataset

In [3]:
data class BBox(val xmin: Double, val ymin: Double, val xmax: Double, val ymax: Double)
class Geometry(val type: String, val coordinates: List<Any>)
class FeatureCollection(val type: String, val properties: Map<String, Any>, val geometry: Geometry)
class GeoJSON(val type: String, val features: List<FeatureCollection>)

In [4]:
fun getSpatialData(geoJSONData: String): SpatialDataset {
    val parsedGeoJson: GeoJSON = Klaxon().parse<GeoJSON>(geoJSONData) ?: throw Exception("Parsing error")
    val geometries = parsedGeoJson.features.map { feature -> Klaxon().toJsonString(feature.geometry) }
    val properties = parsedGeoJson.features.map { feature -> feature.properties }
    val propertiesData = if (properties.isEmpty()) {
        emptyMap()
    } else {
        properties[0].keys.map { col: String -> col to properties.map { record -> record[col] } }.toMap()
    }
    return SpatialDataset.withGEOJSON(data = propertiesData, geometry = geometries)
}

Line_12.jupyter.kts (1:42 - 56) Unresolved reference: SpatialDataset
Line_12.jupyter.kts (10:12 - 26) Unresolved reference: SpatialDataset

In [5]:
val kotlinBBox = BBox(29.63, 59.965, 29.815, 60.035)

#### Load boundaries of St.-Petersburg districts.

In [6]:
val spbDistrictsURL = "https://raw.githubusercontent.com/JetBrains/lets-plot-docs/master/data/spb_districts.geojson"
val spbDistrictsData = URL(spbDistrictsURL).readText()
val spbDistrictsSpatialData = getSpatialData(spbDistrictsData)

Line_14.jupyter.kts (3:31 - 45) Unresolved reference: getSpatialData

#### Create a map showing all districts of St.-Petersburg.

This map will become the **inset map**. The red rectangle indicates the bounds of the future **main map**.

In [7]:
val spbPlot = letsPlot() +
    geomPolygon(map = spbDistrictsSpatialData, color = "#a1d99b", fill = "#f7fcf5") +
    geomRect(xmin = kotlinBBox.xmin, ymin = kotlinBBox.ymin, xmax = kotlinBBox.xmax, ymax = kotlinBBox.ymax, color = "red", alpha = 0) +
    geomText(label = "Saint Petersburg", x = 30.334445, y = 59.934294, color = "black", size = 6) +
    scaleXContinuous(expand = listOf(0.0, 0.0)) + scaleYContinuous(expand = listOf(0.0, 0.0)) +
    themeVoid() + theme(panelBackground = elementRect(color = "black", fill = "white"))
spbPlot

Line_15.jupyter.kts (2:23 - 46) Unresolved reference: spbDistrictsSpatialData

#### Create the main map with only Kotlin island on it.

We use `xlim` and `ylim` parameters of the coordinate system to crop the entire map containing all districts of St.-Petersburg.

In [8]:
// SpatialDataset containing names and coordinates of some tourist attractions to show on the main map
val kotlinPlacesURL = "https://raw.githubusercontent.com/JetBrains/lets-plot-docs/master/data/kotlin_places.geojson"
val kotlinPlacesData = URL(kotlinPlacesURL).readText()
val kotlinPlacesSpatialData = getSpatialData(kotlinPlacesData)

Line_16.jupyter.kts (4:31 - 45) Unresolved reference: getSpatialData

In [9]:
// Cut-out the Kotlin area and add layes with text and points of interest.
val kotlinPlot = letsPlot() +
    geomRect(xmin = kotlinBBox.xmin, ymin = kotlinBBox.ymin, xmax = kotlinBBox.xmax, ymax = kotlinBBox.ymax, fill = "#aadaff", alpha = 0.2) +
    geomPolygon(map = spbDistrictsSpatialData, color = "#31a354", fill = "#e5f5e0") +
    geomPoint(map = kotlinPlacesSpatialData, size = 5) { color = "type"; shape = "type" } +
    geomText(map = kotlinPlacesSpatialData, hjust = "right", position = positionNudge(x = -.002)) { label = "name" } +
    geomText(label = "Kotlin Isl.", x = 29.725, y = 60.011, color = "#31a354", size = 13, fontface = "italic") +
    geomText(label = "Gulf of Finland", x = 29.665, y = 60.002, color = "#578bcc", size = 11, fontface = "italic") +
    coordCartesian(xlim = Pair(kotlinBBox.xmin, kotlinBBox.xmax), ylim = Pair(kotlinBBox.ymin, kotlinBBox.ymax)) +
    ggtitle("Tourist attractions on Kotlin island") +
    themeVoid() + theme().legendPosition(.15, .2)
kotlinPlot

Line_17.jupyter.kts (4:23 - 46) Unresolved reference: spbDistrictsSpatialData
Line_17.jupyter.kts (5:21 - 44) Unresolved reference: kotlinPlacesSpatialData
Line_17.jupyter.kts (6:20 - 43) Unresolved reference: kotlinPlacesSpatialData

#### Finally, use `GGBunch` to show these two maps together.

In [10]:
val bunch = GGBunch()
bunch.addPlot(kotlinPlot, 0, 0, 800, 600)
bunch.addPlot(spbPlot, 600, 25, 200, 150)
bunch.show()

Line_18.jupyter.kts (2:15 - 25) Unresolved reference: kotlinPlot
Line_18.jupyter.kts (3:15 - 22) Unresolved reference: spbPlot